<div id="colab_button">
  <h1>Quick tour</h1>
  <a target="_blank" href="https://colab.research.google.com/github/mithril-security/blindbox/blob/main/docs/docs/getting-started/quick-tour.ipynb"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>
______________________________

## Introduction
______________________________

**BlindBox** protects SaaS users' data. It wraps applications in a sandbox with customizable access possibilities and deploys them on **Confidential VMs**, which provide **robust hardware based isolation and protection**. When clients connect to a BlindBox and query the application with their data, the service provider has no way of accessing or seeing this data - hence our name BlindBox! 

Using BlindBox happens in 4 steps:

![workflow_steps.png](../../assets/workflow_4_steps.png)

## Quick start guide
______________________

We'll use an image containing [OpenAI's Whisper model](https://huggingface.co/spaces/openai/whisper) as the example software to walk you through how you can use BlindBox.

We made a video demo, but if you want to try the code for yourself, all the steps are just under. 👇

***WE MADE A VIDEO!***

***[TO DO: VIDEO DEMO]***

### Pre-requisites

To follow along with this guide and deploy your own BlindBox, you will need:

+ An **Azure account** and **credits**. 

	> **Important**: Our quick tour deploys a Whisper app on a DCasv5/ECasv5 series confidential VM with Azure. If you follow along with the Quick Tour, you will be charged according to your usage as detailed by Azure. You can get a detailed pricing quote [here](https://learn.microsoft.com/en-us/azure/confidential-computing/confidential-vm-overview).

+ Have Docker installed in your environment. Here's the [Docker installation guide](https://docs.docker.com/desktop/install/linux-install/).

#### Installing BlindBox and Terraform

Let's dive in! We'll first need to install the **[blindbox](https://pypi.org/project/blindbox/) PyPi package** and **`terraform`**, which will facilitate the deployment of our BlindBox.

In [ ]:
# install blindbox
!pip install blindbox

# Download terraform
!wget -O- https://apt.releases.hashicorp.com/gpg | sudo gpg --dearmor -o /usr/share/keyrings/hashicorp-archive-keyring.gpg

# Add terraform to trusted apt packages
!echo "deb [signed-by=/usr/share/keyrings/hashicorp-archive-keyring.gpg] https://apt.releases.hashicorp.com $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/hashicorp.list

# Install terraform
!sudo apt update && sudo apt install terraform

#### Azure CLI and log in

We will also need to download the **Azure CLI tool** and **log in** to our Azure account.

In [ ]:
# Install azure-cli
!curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

# Install confcom extension for azure-cli
!az extension add -n confcom

# Log in using azure-cli
!az login

# Set the subscription azure-cli will use by default.
!az account set --subscription "Microsoft Azure Sponsorship"

When running the `az login` command, you will be provided with a link that you will need to open in your browser to successfully login to your Azure account.

#### Download the audio file

The final requirement is to download our [test audio file](https://www2.cs.uic.edu/~i101/SoundFiles/taunt.wav) which we will use to query our Whisper model. This one was provided by University of Illinois at Chicago  as part of their Computer Science 101 course.

In [ ]:
# Download our example audio file and save it as `test.wav` in a `.tmp` folder

!mkdir -p .tmp
!wget https://www2.cs.uic.edu/~i101/SoundFiles/taunt.wav -O .tmp/test.wav

### STEP 1. Preparing the application image

In BlindBox architecture, the "inner" application image will be wrapped within an "outer" BlindBox image. The outer image is responsible for implementing customizable security policy and **communicates** with the inner image **using port 80**.

This is why, to be compatible with BlindBox, applications images only need to fill one requirement: they must expose `port 80`. 

You can download the Docker image we prepared for this tutorial in the `ai_server_example` folder of the `BlindBox repository`:

In [ ]:
!git clone https://github.com/mithril-security/blindbox
!cd ai_server_example

We can now build our Whisper application image so it is ready to be deployed with BlindBox.

In [ ]:
# build Whisper application assigning it the tag "myimage"
!docker build -t myimage .

### STEP 2. Wrapping the application in an enclave

#### Preparing the BlindBox

The next step is to initialize our BlindBox. This step will create two template configuration files needed for our BlindBox, which users can modify to apply custom settings and policies: 

+ A `blindbox.yml` file which has modifiable options. It should be modified before building your BlindBox for changes to take effect. 
	+ `platform` to specify the TEE type you want to use (we currently only support `amd-sev`)
	+ `ip-rules` to whitelist the `ip addresses` of the users allowed to query the BlindBox application. 

+ A `blindbox.tf` file. It contains all the configuration options for the Confidential VM which we deploy our BlindBox image on. It should be modified before deploying your BlindBox for changes to take effect.

To perform this initialization step, we run the following command:

In [ ]:
# initialize the blindbox
!blindbox --platform azure-sev init 

We specify that we want to use the AMD SEV TEE platform with Azure using the `platform` option with `azure-sev` as our argument.

> Note this is the only platform currently supported. `aws-nitro` is **coming soon**.

#### Building the BlindBox image

Next we are ready to build our BlindBox image which is a wrapper around your application image. It implements the custom security policies specified in the `blindbox.yml` file.

We do this using `blindbox build` and specifying the application image using the `--source-image` option.

In [ ]:
# build the application image with the tag "myimage-blindbox:v1"
!blindbox build -t myimage-blindbox:v1 --source-image myimage:v1

### STEP 3. Deploy the BlindBox image on Confidential VM

We next use the `deploy` command to deploy our `blindbox image` (which contains the inner `application image`) on an `Azure confidential VM`. 

In [ ]:
# deploy our image inside Confidential VM using BlindBox
!blindbox deploy

### STEP 4. Querying the enclave

Now that the deployment is done, authorised users can query the model within the enclave using the `requests` module within the `blindbox` library we installed earlier on.

In [ ]:
# import requests module
from blindbox  import requests

# we query our application, sending it our audio file
res = requests.post(url="http://localhost:8080/enclave/predict", files={"audio": open(".tmp/test.wav", "rb")})

# display result
print(res)

Here, we were able to transcribe our audio file while keeping the audio file confidential, even from the SaaS vendor!

## Why trust us?
___________________________________________

+ **Our core security features are open source.** We believe that transparency is the best way to ensure security and you can inspect the code yourself on our [GitHub page](https://github.com/mithril-security/blindbox).

+ **Our historical project [BlindAI](../past-projects/blindai.md) was successfully audited** by Quarkslab. Although both projects differ (BlindAI was meant for the confidential deployment of ONNX models inside Intel SGX enclaves), we want to highlight that we are serious about our security standards and know how to code secure remote attestation. 

+ BlindBox is still under development, but here's what will happen when the enclave is not secure. 

	If we talk to a server with no collateral to prove it is a secure enclave, which means that no signature from the hardware provider, then an error will be raised:

	```python
	import blindbox.requests as requests

	session = requests.SecureSession()

	>> NotAnEnclaveError
	```

	If the first check passes, but the hash of the code is not the publicly available hash of BlindBox, another error will be raised:

	```python
	import blindbox.requests as requests

	session = requests.SecureSession()

	>> InvalidEnclaveCode
	```

	Here, ***WE'RE NOT RESPECTING THE CUSTOM POLICY REQUIREMENTS***

	```python
	import blindbox.requests as requests

	session = requests.SecureSession()

	>> NotIsolatedEnclaveError
	```

## Conclusion
_______________________
 
Now that we've seen how we can deploy and query an application with BlindBox, let's find out more about how BlindBox protects end users' data in the following [section](./confidential_computing.md).